<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
# !wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [2]:
from gensim.models.keyedvectors import KeyedVectors

wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
# help(wv_embeddings)

In [5]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


In [6]:
wv_embeddings[0][:10]

array([-1.3247741 ,  2.4254758 , -0.48980346,  2.9409528 , -3.1995323 ,
        1.9706817 ,  2.7333171 ,  0.67988634, -0.77023953, -2.5330846 ],
      dtype=float32)

In [7]:
wv_embeddings.index_to_key[0]

'use'

Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слово `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [8]:
# method most_simmilar
'''your code'''

'your code'

In [9]:
# help(wv_embeddings.most_similar)

In [10]:
top30 = wv_embeddings.most_similar(positive=['dog'], negative=[], topn=30)

In [11]:
for i, (word, similarity) in enumerate(top30):
    if word == 'cat':
        print(i, word, similarity)

25 cat 0.6852341294288635


#### Ответ 1:
* 25

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [12]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize, sent_tokenize

text = "All work and no play makes jack a dull boy, all work and no play"
tokenizer = WordPunctTokenizer()  # создаем экземпляр токенизатора
tokens = tokenizer.tokenize(text)  # разбиваем текст на токены

print(tokens)  # выводим результат

['All', 'work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', ',', 'all', 'work', 'and', 'no', 'play']


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
import spacy

nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

class MyTokenizer:
    def __init__(self):
        self.lemmatization = spacy.load('en_core_web_sm')
        self.stopWords = set(stopwords.words('english'))
        self.punctuation = string.punctuation
        
    def tokenize(self, text):
        # return re.findall('\w+', text)
        text = text.lower()
        # лемматизация
        spacy_results = self.lemmatization(text)
        text = [token.lemma_ for token in spacy_results]
        # print(text)
        #cтоп слова 
        text = [word for word in text if word not in self.stopWords]
        # пунктуация
        text = [word for word in text if word not in self.punctuation]
        # print(text)
        text = ' '.join(text)
        # токенизация       
        tokens = word_tokenize(text)
        #bigram = list(nltk.ngrams(tokens, 2))
        #print('bigram = ', bigram)
        return tokens

text = 'My neighbour often drills. This noise doesn’t let me concentrate on my homework!'

tokenizer = MyTokenizer()
tokenizer.tokenize(text)

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-03-28 19:00:54.834698: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 19:00:54.877809: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 19:00:55.743001: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-28 19:00:56.493307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call

['neighbour', 'often', 'drill', 'noise', 'let', 'I', 'concentrate', 'homework']

In [14]:
import numpy as np 


def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    tokens = tokenizer.tokenize(question)
    # print(tokens)
    vecs = np.zeros(dim)
    k = 0
    for token in tokens:
        if token in embeddings:
            k += 1
            vecs += embeddings[token]
    if k == 0:
        return vecs
    return vecs / k

question_to_vec('', wv_embeddings, tokenizer)[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
question_to_vec('I love neural networks?', wv_embeddings, tokenizer)[:10]

array([-1.25681178, -1.26777713, -1.40681175, -0.90633385,  0.37728501,
       -0.4538045 ,  1.22090296, -0.3721061 ,  0.43070222,  0.29966467])

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [16]:
'''your code'''

sentence = 'I love neural networks'
tokens = tokenizer.tokenize(sentence)
print(tokens)

vector = question_to_vec(sentence, wv_embeddings, tokenizer)
print(f'{vector[2]:.2f}')

['I', 'love', 'neural', 'network']
-1.41


#### Ответ 2:
* Если правильно поняла, -1.41

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K

Hits@K - это метрика, используемая в оценке качества ранжирования информационного поиска. Она измеряет долю правильных предсказаний среди топ-K элементов, которые были предложены моделью или системой.Hits@K - это метрика, используемая в оценке качества ранжирования информационного поиска. Она измеряет долю правильных предсказаний среди топ-K элементов, которые были предложены моделью или системой.

Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K

DCG@K (Discounted Cumulative Gain at K) - это метрика, используемая для оценки качества ранжирования результатов поиска или рекомендаций. Она измеряет полезность и порядок представления элементов в списке. 

Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Ответ 3:
* Максимум `Hits@47 - DCG@1 = 1 - 0 = 1`

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$

$$ \text{DCG@K} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2(10)}$$

In [17]:
np.log2(10) 

3.321928094887362

#### Ответ 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$ равен 3.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [18]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    N = len(dup_ranks)
    indicator = [dup_ranks[i] <= k for i in range(N)]
    hits_value = sum(indicator) / N
    return hits_value    

In [19]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    N = len(dup_ranks)
    S = sum([int(dup_ranks[i] <= k) / np.log2(1 + dup_ranks[i]) for i in range(N)])
    dcg_value = S / N
    return dcg_value

In [20]:
N = 1
R = 3
dup_ranks = [2]

K = 1
print(f'hits_count(k={K}) = {hits_count(dup_ranks, K)} ,  \
dcg_score(k={K}) = {dcg_score(dup_ranks, K)}')

K = 4
print(f'hits_count(k={K}) = {hits_count(dup_ranks, K)} , \
dcg_score(k={K}) = {dcg_score(dup_ranks, K)}')

hits_count(k=1) = 0.0 ,  dcg_score(k=1) = 0.0
hits_count(k=4) = 1.0 , dcg_score(k=4) = 0.6309297535714575


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [21]:
import pandas as pd

In [22]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

In [23]:
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

In [24]:
# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [25]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [26]:
# !unzip stackoverflow_similar_questions.zip

Считайте данные.

In [27]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [28]:
validation_data = read_corpus('data/validation.tsv')

In [29]:
# validation_data[0]

Кол-во строк

In [30]:
len(validation_data)

3760

Размер нескольких первых строк

In [31]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


In [32]:
# validation_data[0][:500]

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [33]:
'''your code'''

sentence = 'I love neural networks'
tokens = tokenizer.tokenize(sentence)
print(tokens)

vector = question_to_vec(sentence, wv_embeddings, tokenizer)
print(f'{vector[2]:.2f}')

['I', 'love', 'neural', 'network']
-1.41


In [34]:
# сортируем список кандидатов с учетом позиции
# sorted_candidates = sorted(candidates, key=lambda candidate: cosine_similarity(candidate, question), reverse=True)

In [35]:
# Создание массива
arr = np.array([
    [1, 4, 7],
    [3, 6, 1],
    [2, 8, 5]
])

# Сортировка по третьему столбцу
sorted_arr = arr[arr[:, 2].argsort()]

print(sorted_arr)

[[3 6 1]
 [2 8 5]
 [1 4 7]]


In [36]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    # 1 - cosine_similarity
   
    vec_question = question_to_vec(question, embeddings, tokenizer)
    # print(vec_question[:10])
    
    len_candidates = len(candidates)
    vec_candidates = np.zeros((len_candidates, dim))
    for i in range(len_candidates):
        # print(candidates[i])
        vec_candidates[i] = question_to_vec(candidates[i], embeddings, tokenizer)
        
    
    norm_vec_question = np.linalg.norm(vec_question, ord=2)

    сos_dist = np.zeros(len_candidates)
    for i in range(len_candidates):
        norm_vec_candidat = np.linalg.norm(vec_candidates[i], ord=2)
        if norm_vec_candidat != 0: 
            cos_similarity = vec_question @ vec_candidates[i] / (norm_vec_question * norm_vec_candidat)  
        else:
            cos_similarity = 0
        сos_dist[i] = 1 - cos_similarity
        
    ind = np.arange(сos_dist.shape[0]).reshape(-1, 1)    
    ind_сos_dist = np.c_[ind, сos_dist]
    sort_ind_сos_dist = ind_сos_dist[ind_сos_dist[:, 1].argsort()]
    
    result = [(int(ind), candidates[int(ind)]) for ind, _ in sort_ind_сos_dist]
    
    return result
    
    

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [37]:
questions = ['converting string to list', 
             'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [38]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    print(*ranks, sep='\n')
    print()

(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')
(1, 'WPF- How to update the changes in list item of a list')



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [39]:
# должно вывести
# results = [[(1, 'C# create cookie from string and send it'),
#             (0, 'Convert Google results object (pure js) to Python object'),
#             (2, 'How to use jQuery AJAX for an outside domain?')],
#            [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
#             (*, 'select2 not displaying search results'), #скрыт
#             (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

#### Ответ 5:
* 123

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [41]:
from tqdm.notebook import tqdm

In [42]:
1000 * 5 / 3600 # часов все это выполняется

1.3888888888888888

In [43]:
%%time
# невозможно долго
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data[:15])):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1) # смотрим на какое место встал первый кандидат

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 1min 6s, sys: 576 ms, total: 1min 6s
Wall time: 1min 6s


In [44]:
wv_ranking

[2, 32, 3, 1, 9, 2, 2, 5, 1, 1, 546, 1, 386, 1, 1]

In [46]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.400 | Hits@   1: 0.400
DCG@   5: 0.585 | Hits@   5: 0.733
DCG@  10: 0.605 | Hits@  10: 0.800
DCG@ 100: 0.619 | Hits@ 100: 0.867
DCG@ 500: 0.626 | Hits@ 500: 0.933
DCG@1000: 0.634 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [47]:
train_data = read_corpus('./data/train.tsv')

In [ ]:
# validation_data[:3]

In [48]:
len(train_data)

1000000

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
# размер окна взала 3 и 5
# По результатом качество не на много улучшилось потому что я убрала стоп слова и пунктуацию

In [49]:
train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object\n']

In [50]:
texts = [' '.join(train_data[i]) for i in range(len(train_data))]
len(texts)

1000000

In [51]:
%%time 

words = []
for text in tqdm(texts[:50000]):
    words.append(tokenizer.tokenize(text))
print(words[0])

  0%|          | 0/50000 [00:00<?, ?it/s]

['convert', 'string', 'list', 'convert', 'google', 'result', 'object', 'pure', 'js', 'python', 'object']
CPU times: user 4min 48s, sys: 3.1 s, total: 4min 51s
Wall time: 4min 46s


In [52]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,     # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=5, # игнорирует все слова с общей частотой ниже этого значения.
                              window=3).wv #это размер окна

In [53]:
%%time 

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data[:15])):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
wv_ranking

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.400 | Hits@   1: 0.400
DCG@   5: 0.400 | Hits@   5: 0.400
DCG@  10: 0.419 | Hits@  10: 0.467
DCG@ 100: 0.482 | Hits@ 100: 0.800
DCG@ 500: 0.508 | Hits@ 500: 1.000
DCG@1000: 0.508 | Hits@1000: 1.000
CPU times: user 1min 6s, sys: 612 ms, total: 1min 6s
Wall time: 1min 5s


In [54]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,     # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=5, # игнорирует все слова с общей частотой ниже этого значения.
                              window=5).wv #это размер окна

In [55]:
%%time 

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data[:15])):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
wv_ranking

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.400 | Hits@   1: 0.400
DCG@   5: 0.400 | Hits@   5: 0.400
DCG@  10: 0.400 | Hits@  10: 0.400
DCG@ 100: 0.495 | Hits@ 100: 0.867
DCG@ 500: 0.513 | Hits@ 500: 1.000
DCG@1000: 0.513 | Hits@1000: 1.000
CPU times: user 1min 5s, sys: 678 ms, total: 1min 6s
Wall time: 1min 5s


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Word2Vec дает лучшие результаты потому что
2. В данном случае - да
3. Полученные Word2Vec, потому что обучены на тестовых данных (именно под этот датасет)
4. Вероятно, потому что нужны другие алгоритмы
5. Использовать RNN )

Еще очень долго работает. Хочется структур побыстрее. 